In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

from sklearn.preprocessing import LabelEncoder, StandardScaler
from torch.utils.data import Dataset, DataLoader

## Load the train dataset

In [3]:
root = '../KuaiRec 2.0/'

# Training data
train = pd.read_csv(root + "data_exports/joined_train_data_segmented.csv")

print(f'Total number of training data: {len(train)}')

Total number of training data: 2552082


In [3]:
train.head()

,user_id,video_id,time,watch_ratio,user_active_degree,is_lowactive_period,is_live_streamer,is_video_author,follow_user_num,fans_user_num,...,avg_daily_watch_time,top_3_categories,cluster,News_Politics,Auto_Tech,Lifestyle,Sports_Fitness,Entertainment,Culture,Others
0,14,148,2020-07-05 05:27:48.378,0.722103,full_active,0,0,1,73,6,...,8.360719e+12,"['Car', 'Pets', 'Real estate家居']",0,0,1,1,0,0,0,1
1,14,183,2020-07-05 05:28:00.057,1.907377,full_active,0,0,1,73,6,...,8.360719e+12,"['Car', 'Pets', 'Real estate家居']",0,0,1,1,0,0,0,1
2,14,3649,2020-07-05 05:29:09.479,2.063311,full_active,0,0,1,73,6,...,8.360719e+12,"['Car', 'Pets', 'Real estate家居']",0,0,1,1,0,0,0,1
3,14,5262,2020-07-05 05:30:43.285,0.566388,full_active,0,0,1,73,6,...,8.360719e+12,"['Car', 'Pets', 'Real estate家居']",0,0,1,1,0,0,0,1
4,14,8234,2020-07-05 05:35:43.459,0.418364,full_active,0,0,1,73,6,...,8.360719e+12,"['Car', 'Pets', 'Real estate家居']",0,0,1,1,0,0,0,1


### Get the current date
This is necessary to calculate the number of days since the last interaction as well as the age of the videos, which will be used for the time decay component of our model.
We assume it to be the day of the latest interaction in the training data. 

In [31]:
# Convert type to datetime
train['time'] = pd.to_datetime(train['time'])

# Assume current date is the next day of the last date in the training data
CURRENT_DATE = train['time'].dt.date.max()

# Just the date portion
print(f'Current date: {CURRENT_DATE}')

Current date: 2020-08-03


### Calculate age of video

In [23]:
video_info = pd.read_csv(root + 'data/item_daily_features.csv', usecols=['video_id', 'upload_dt']).drop_duplicates()

video_info['upload_dt'] = pd.to_datetime(video_info['upload_dt'])

In [35]:
# Get videos in the training data
video_info_filtered = video_info[video_info['video_id'].isin(train['video_id'].unique())]

# Calculate video age
# video_info_filtered['Current Date'] = CURRENT_DATE
video_info_filtered['video_age'] = (CURRENT_DATE - video_info_filtered['upload_dt'].dt.date).dt.days

# Convert into dictionary
video_age_dict = video_info_filtered.set_index('video_id')['video_age'].to_dict()
video_age_dict

C:\Users\chong\AppData\Local\Temp\ipykernel_18532\2834565506.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  video_info_filtered['video_age'] = (CURRENT_DATE - video_info_filtered['upload_dt'].dt.date).dt.days


{103: 38,
 109: 37,
 120: 36,
 122: 35,
 128: 34,
 130: 33,
 131: 33,
 133: 33,
 136: 33,
 137: 33,
 139: 33,
 142: 33,
 145: 32,
 146: 32,
 147: 32,
 148: 32,
 151: 31,
 152: 31,
 153: 31,
 154: 31,
 164: 30,
 166: 30,
 168: 30,
 169: 30,
 170: 30,
 171: 30,
 173: 30,
 175: 30,
 179: 30,
 180: 30,
 183: 30,
 186: 30,
 188: 30,
 203: 29,
 206: 29,
 207: 29,
 210: 29,
 211: 29,
 217: 29,
 223: 28,
 229: 28,
 237: 28,
 238: 28,
 250: 28,
 251: 28,
 254: 28,
 255: 28,
 256: 28,
 258: 28,
 262: 28,
 265: 28,
 267: 28,
 270: 28,
 272: 28,
 275: 27,
 279: 27,
 280: 27,
 282: 27,
 285: 27,
 286: 27,
 288: 27,
 289: 27,
 290: 27,
 296: 27,
 297: 27,
 300: 27,
 302: 27,
 304: 27,
 306: 27,
 307: 27,
 314: 27,
 318: 27,
 319: 27,
 324: 26,
 331: 26,
 336: 26,
 340: 26,
 349: 26,
 350: 26,
 351: 26,
 361: 25,
 364: 25,
 365: 25,
 368: 25,
 369: 25,
 372: 25,
 373: 25,
 378: 25,
 388: 25,
 389: 25,
 390: 25,
 391: 25,
 392: 25,
 394: 25,
 395: 25,
 400: 25,
 402: 25,
 403: 25,
 408: 24,
 413: 24,


### Calculate age of the interaction

In [66]:
# Get interaction age as dictionary
interaction = train[['user_id', 'video_id', 'time']]
interaction['interaction_age'] = (CURRENT_DATE - interaction['time'].dt.date).dt.days

interaction_age_dict = interaction.groupby(['user_id', 'video_id'])['interaction_age'].max().to_dict()
interaction_age_dict

C:\Users\chong\AppData\Local\Temp\ipykernel_18532\3890013064.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  interaction['interaction_age'] = (CURRENT_DATE - interaction['time'].dt.date).dt.days


{(14, 103): 26,
 (14, 109): 25,
 (14, 120): 25,
 (14, 122): 25,
 (14, 128): 25,
 (14, 130): 8,
 (14, 131): 13,
 (14, 133): 14,
 (14, 136): 21,
 (14, 137): 13,
 (14, 139): 28,
 (14, 142): 13,
 (14, 145): 27,
 (14, 146): 19,
 (14, 147): 27,
 (14, 148): 29,
 (14, 151): 4,
 (14, 153): 13,
 (14, 154): 14,
 (14, 164): 22,
 (14, 166): 16,
 (14, 168): 20,
 (14, 169): 24,
 (14, 170): 22,
 (14, 171): 29,
 (14, 173): 13,
 (14, 175): 29,
 (14, 179): 29,
 (14, 180): 26,
 (14, 183): 29,
 (14, 186): 29,
 (14, 188): 22,
 (14, 203): 28,
 (14, 206): 28,
 (14, 207): 20,
 (14, 210): 24,
 (14, 211): 28,
 (14, 217): 28,
 (14, 223): 27,
 (14, 229): 27,
 (14, 237): 17,
 (14, 238): 18,
 (14, 250): 6,
 (14, 251): 14,
 (14, 254): 27,
 (14, 255): 14,
 (14, 256): 27,
 (14, 258): 27,
 (14, 262): 27,
 (14, 265): 27,
 (14, 267): 18,
 (14, 270): 16,
 (14, 272): 24,
 (14, 275): 27,
 (14, 279): 27,
 (14, 280): 27,
 (14, 282): 25,
 (14, 285): 27,
 (14, 286): 27,
 (14, 288): 26,
 (14, 289): 27,
 (14, 290): 27,
 (14, 296):

In [36]:
train.columns

Index(['user_id', 'video_id', 'time', 'watch_ratio', 'user_active_degree',
       'is_lowactive_period', 'is_live_streamer', 'is_video_author',
       'follow_user_num', 'fans_user_num', 'friend_user_num', 'register_days',
       'author_id', 'video_type', 'video_tag_name', 'video_duration',
       'show_cnt', 'play_cnt', 'play_duration', 'like_cnt', 'comment_cnt',
       'share_cnt', 'follow_cnt', 'collect_cnt', 'manual_cover_text',
       'caption', 'topic_tag', 'first_level_category_name',
       'second_level_category_name', 'third_level_category_name',
       'english_caption', 'english_first_level_category_name',
       'english_second_level_category_name',
       'english_third_level_category_name', 'english_topic_tag', 'is_new_user',
       'total_connections', 'is_content_creator', 'hour', 'day_of_week',
       'watch_frequency', 'is_weekend_interaction', 'is_weekend',
       'time_period', 'count_afternoon_views', 'count_evening_views',
       'count_midnight_views', 'count_m

In [ ]:
# Segregate by 
# 'cluster'

# One hot encode
# 'user_active_degree', 'time_period'

# Remove
# 'author_id', 'video_type' (all 'NORMAL'), 
# 'manual_cover_text', 'video_tag_name'
# 'caption', 'topic_tag', 'first_level_category_name',
# 'second_level_category_name', 'third_level_category_name',
# 'english_caption', 'english_first_level_category_name',
# 'english_second_level_category_name',
# 'english_third_level_category_name', 'english_topic_tag'
# 'top_3_categories' (use the OHE categories)
# 'play_duration' (as it is highly correlated to target variable 'watch_ratio' (watch_ratio = play_duration / video_duration))

# To incorporate (time decay)
# 'time', 'video_age'

## Preprocessing for feeding into Neural Network portion of NCF

### One hot encode categorical variables

In [67]:
# One hot encode 'user_active_degree', 'time_period'
train_processed = pd.get_dummies(train, columns=['user_active_degree', 'time_period'])

# Remove the column for user_active_degree = UNKNOWN
train_processed = train_processed.drop(columns=['user_active_degree_UNKNOWN'])

In [68]:
train_processed = train_processed.drop(columns=['author_id', 'video_type', 
                                    'video_tag_name', 
                                    'manual_cover_text', 'caption', 'topic_tag', 
                                    'first_level_category_name', 'second_level_category_name', 'third_level_category_name',
                                    'english_caption', 'english_first_level_category_name',
                                    'english_second_level_category_name',
                                    'english_third_level_category_name', 'english_topic_tag',
                                    'top_3_categories',
                                    'play_duration'
                                    ])

### Scale continuous variables

In [69]:
train_processed.columns

Index(['user_id', 'video_id', 'time', 'watch_ratio', 'is_lowactive_period',
       'is_live_streamer', 'is_video_author', 'follow_user_num',
       'fans_user_num', 'friend_user_num', 'register_days', 'video_duration',
       'show_cnt', 'play_cnt', 'like_cnt', 'comment_cnt', 'share_cnt',
       'follow_cnt', 'collect_cnt', 'is_new_user', 'total_connections',
       'is_content_creator', 'hour', 'day_of_week', 'watch_frequency',
       'is_weekend_interaction', 'is_weekend', 'count_afternoon_views',
       'count_evening_views', 'count_midnight_views', 'count_morning_views',
       'avg_daily_watch_time', 'cluster', 'News_Politics', 'Auto_Tech',
       'Lifestyle', 'Sports_Fitness', 'Entertainment', 'Culture', 'Others',
       'user_active_degree_full_active', 'user_active_degree_high_active',
       'user_active_degree_middle_active', 'time_period_afternoon',
       'time_period_evening', 'time_period_midnight', 'time_period_morning'],
      dtype='object')

In [70]:
# We can see that the values are all on different scales. For example, follow_user_num is in the tens-thousands while like_cnt can range form millions to billions. This will affect the training of the model, therefore scaling is needed

train_processed[['follow_user_num',
       'fans_user_num', 'friend_user_num', 'register_days', 'video_duration',
       'show_cnt', 'play_cnt', 'like_cnt', 'comment_cnt',
       'share_cnt', 'follow_cnt', 'collect_cnt', 'count_afternoon_views', 'count_evening_views', 'count_midnight_views',
       'count_morning_views', 'avg_daily_watch_time']].describe()

,follow_user_num,fans_user_num,friend_user_num,register_days,video_duration,show_cnt,play_cnt,like_cnt,comment_cnt,share_cnt,follow_cnt,collect_cnt,count_afternoon_views,count_evening_views,count_midnight_views,count_morning_views,avg_daily_watch_time
count,2.552082e+06,2.552082e+06,2.552082e+06,2.552082e+06,2.552082e+06,2.552082e+06,2.552082e+06,2.552082e+06,2.552082e+06,2.552082e+06,2.552082e+06,2.552082e+06,2.552082e+06,2.552082e+06,2.552082e+06,2.552082e+06,2.552082e+06
mean,5.381411e+01,3.872561e+00,1.331606e+00,2.653341e+02,1.164791e+04,6.959049e+06,7.052437e+06,2.044780e+05,8.935899e+03,3.805251e+03,2.093272e+04,2.858760e+02,4.658341e+02,2.809108e+02,4.579366e+02,6.100598e+02,8.062631e+12
std,1.418902e+02,9.716679e+00,4.924868e+00,2.640708e+02,1.344116e+04,9.275605e+06,9.511481e+06,3.209431e+05,2.111983e+04,1.269530e+04,6.331006e+04,1.337505e+03,2.844922e+02,2.385123e+02,4.339834e+02,3.305712e+02,7.068827e+11
min,0.000000e+00,0.000000e+00,0.000000e+00,8.000000e+00,3.066000e+03,6.440000e+02,3.310000e+02,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.632392e+12
25%,7.000000e+00,0.000000e+00,0.000000e+00,1.190000e+02,7.333000e+03,8.329130e+05,7.629220e+05,1.552800e+04,3.450000e+02,6.400000e+01,1.002000e+03,5.000000e+00,2.490000e+02,8.400000e+01,5.300000e+01,3.740000e+02,7.686325e+12
50%,1.500000e+01,1.000000e+00,0.000000e+00,2.000000e+02,9.383000e+03,3.127692e+06,3.071419e+06,7.359000e+04,2.171000e+03,4.140000e+02,4.968000e+03,2.800000e+01,4.440000e+02,2.250000e+02,3.560000e+02,5.690000e+02,8.158000e+12
75%,4.300000e+01,4.000000e+00,1.000000e+00,3.020000e+02,1.150000e+04,9.372330e+06,9.544620e+06,2.512090e+05,8.918000e+03,2.275000e+03,1.797800e+04,1.330000e+02,6.560000e+02,4.190000e+02,7.480000e+02,8.060000e+02,8.518700e+12
max,1.811000e+03,2.510000e+02,7.100000e+01,2.002000e+03,2.945200e+05,6.525508e+07,6.479578e+07,2.762854e+06,3.383650e+05,2.061050e+05,1.215372e+06,2.919700e+04,1.477000e+03,1.435000e+03,1.852000e+03,1.727000e+03,1.277244e+13


In [71]:
scaler = StandardScaler()

columns_to_scale = ['follow_user_num',
       'fans_user_num', 'friend_user_num', 'register_days', 'video_duration',
       'show_cnt', 'play_cnt', 
       'like_cnt', 'comment_cnt',
       'share_cnt', 'follow_cnt', 'collect_cnt', 
       'total_connections',
       'watch_frequency', 
       'count_afternoon_views', 'count_evening_views', 'count_midnight_views',
       'count_morning_views', 
       'avg_daily_watch_time', 
       ]

train_processed[columns_to_scale] = scaler.fit_transform(train_processed[columns_to_scale])

In [72]:
train_processed[['follow_user_num',
       'fans_user_num', 'friend_user_num', 'register_days', 'video_duration',
       'show_cnt', 'play_cnt', 'like_cnt', 'comment_cnt',
       'share_cnt', 'follow_cnt', 'collect_cnt', 'count_afternoon_views', 'count_evening_views', 'count_midnight_views',
       'count_morning_views', 'avg_daily_watch_time']].describe()

# We now see that the mean of all the columns is (close to) 0 and the standard deviation is 1

,follow_user_num,fans_user_num,friend_user_num,register_days,video_duration,show_cnt,play_cnt,like_cnt,comment_cnt,share_cnt,follow_cnt,collect_cnt,count_afternoon_views,count_evening_views,count_midnight_views,count_morning_views,avg_daily_watch_time
count,2.552082e+06,2.552082e+06,2.552082e+06,2.552082e+06,2.552082e+06,2.552082e+06,2.552082e+06,2.552082e+06,2.552082e+06,2.552082e+06,2.552082e+06,2.552082e+06,2.552082e+06,2.552082e+06,2.552082e+06,2.552082e+06,2.552082e+06
mean,3.875563e-18,-3.162816e-17,1.933327e-17,-6.521637e-17,-2.266133e-16,4.790998e-17,8.174320e-18,-2.525798e-17,-1.905485e-17,-1.854256e-18,-9.449469e-18,1.789107e-17,-5.862346e-17,-3.456824e-17,2.940082e-17,1.021901e-16,-2.135391e-15
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-3.792660e-01,-3.985478e-01,-2.703841e-01,-9.744891e-01,-6.384799e-01,-7.501836e-01,-7.414311e-01,-6.371098e-01,-4.231048e-01,-2.997370e-01,-3.306382e-01,-2.137384e-01,-1.637424e+00,-1.177763e+00,-1.055194e+00,-1.845472e+00,-4.852628e+00
25%,-3.299320e-01,-3.985478e-01,-2.703841e-01,-5.541473e-01,-3.210220e-01,-6.604569e-01,-6.612552e-01,-5.887337e-01,-4.067695e-01,-2.946957e-01,-3.148114e-01,-2.100001e-01,-7.621797e-01,-8.255794e-01,-9.330694e-01,-7.140968e-01,-5.323456e-01
50%,-2.735504e-01,-2.956320e-01,-2.703841e-01,-2.474113e-01,-1.685054e-01,-4.130575e-01,-4.185488e-01,-4.078230e-01,-3.203104e-01,-2.671265e-01,-2.521673e-01,-1.928039e-01,-7.674775e-02,-2.344147e-01,-2.348860e-01,-1.242086e-01,1.349154e-01
75%,-7.621467e-02,1.311554e-02,-6.733294e-02,1.388488e-01,-1.100400e-02,2.601751e-01,2.620184e-01,1.456054e-01,-8.475032e-04,-1.205368e-01,-4.667066e-02,-1.142995e-01,6.684398e-01,5.789608e-01,6.683745e-01,5.927325e-01,6.451838e-01
max,1.238413e+01,2.543333e+01,1.414625e+01,6.576517e+00,2.104522e+01,6.284877e+00,6.070911e+00,7.971434e+00,1.559810e+01,1.593501e+01,1.886651e+01,2.161572e+01,3.554284e+00,4.838701e+00,3.212251e+00,3.378820e+00,6.662791e+00


## Create the Dataset

In [117]:
class KuaiShouDataset(Dataset):
    def __init__(self, data, user_id_col, video_id_col, user_feature_cols, video_feature_cols, watch_ratio_col, interaction_age_dict, video_age_dict):
        self.user_feature_cols = user_feature_cols
        self.video_feature_cols = video_feature_cols

        # Initialise and fit LabelEncoders
        self.user_encoder = LabelEncoder()
        self.video_encoder = LabelEncoder()
        
        self.user_indices = torch.tensor(self.user_encoder.fit_transform(data[user_id_col]), dtype=torch.long)
        self.video_indices = torch.tensor(self.video_encoder.fit_transform(data[video_id_col]), dtype=torch.long)

        # Convert user and video features and watch ratios to tensors
        self.user_features = torch.tensor(data[user_feature_cols].values, dtype=torch.float32)
        self.video_features = torch.tensor(data[video_feature_cols].values, dtype=torch.float32)
        self.watch_ratios = torch.tensor(data[watch_ratio_col].values, dtype=torch.float32)

        # Time related features
        self.interaction_age_dict = interaction_age_dict
        self.video_age_dict = video_age_dict

    def __len__(self):
        return len(self.user_indices)

    def __getitem__(self, idx):
        return self.user_indices[idx], self.video_indices[idx], self.user_features[idx], self.video_features[idx], self.watch_ratios[idx]

    def inverse_transform_user_ids(self, encoded_user_idx):
        """Decode encoded user indices to original user_ids."""
        return self.user_encoder.inverse_transform(encoded_user_idx)
    
    def inverse_transform_video_ids(self, encoded_video_idx):
        """Decode encoded video indices to original video_ids."""
        return self.video_encoder.inverse_transform(encoded_video_idx)
    
    def get_interaction_age(self, user_idx, video_idx):
        """Get days since interaction."""
        user_ids = self.inverse_transform_user_ids(user_idx)
        video_ids = self.inverse_transform_video_ids(video_idx)

        ages = []
        for i in range(len(user_ids)):
            ages.append(self.interaction_age_dict[(user_ids[i], video_ids[i])])
        return torch.tensor(ages, dtype=torch.float32)
    
    def get_video_age(self, video_idx):
        """Get video age."""
        video_ids = self.inverse_transform_video_ids(video_idx)

        ages = []
        for i in range(len(video_idx)):
            ages.append(self.video_age_dict[video_ids[i]])
        return torch.tensor(ages, dtype=torch.float32)

## Time Infused Neural Collaborative Filtering

### Defining the model architecture

In [129]:
class NCF(nn.Module):
    def __init__(self, num_users, num_videos, embedding_dim, num_user_features, num_video_features, dropout, alpha, beta):
        super(NCF, self).__init__()

        # Hyperparameters
        self.dropout = dropout
        self.alpha = alpha
        self.beta = beta
        
        # GMF Components for embeddings
        self.user_embeddings_gmf = nn.Embedding(num_users, embedding_dim)
        self.video_embeddings_gmf = nn.Embedding(num_videos, embedding_dim)

        # MLP Components for embeddings
        self.user_embeddings_mlp = nn.Embedding(num_users, embedding_dim)
        self.video_embeddings_mlp = nn.Embedding(num_videos, embedding_dim)

        # MLP layers for user and video embeddings
        self.fc1_mlp = nn.Linear(2 * embedding_dim, 128)
        self.fc2_mlp = nn.Linear(128, 64)

        # MLP layers for user and video features
        self.user_features_fc = nn.Linear(num_user_features, embedding_dim)
        self.video_features_fc = nn.Linear(num_video_features, embedding_dim)

        # Final layers combining GMF, MLP for embeddings, and additional features
        self.fc1_combined = nn.Linear(embedding_dim + 64 + 2 * embedding_dim, 128)
        self.fc2_combined = nn.Linear(128, 1)

    def forward(self, user_idx, video_idx, user_features, video_features, interaction_days, video_age_days):
        ####### GMF Embedding branch #######
        user_emb_gmf = self.user_embeddings_gmf(user_idx)
        video_emb_gmf = self.video_embeddings_gmf(video_idx)
        gmf_output = user_emb_gmf * video_emb_gmf                                   # dimension: (batch_size, embedding_dim)

        ####### MLP Embedding branch #######
        user_emb_mlp = self.user_embeddings_mlp(user_idx)
        video_emb_mlp = self.video_embeddings_mlp(video_idx)
        mlp_input = torch.cat([user_emb_mlp, video_emb_mlp], dim=-1)                # dimension: (batch_size, 2 * embedding_dim)

        # First fully connected layer with BatchNorm and ReLU
        mlp_output = self.fc1_mlp(mlp_input)
        if self.training:
            mlp_output = nn.BatchNorm1d(128)(mlp_output)
        mlp_output = torch.relu(mlp_output)
        mlp_output = nn.Dropout(self.dropout)(mlp_output)

        # Second fully connected layer with BatchNorm and ReLU
        mlp_output = self.fc2_mlp(mlp_output)
        if self.training:
            mlp_output = nn.BatchNorm1d(64)(mlp_output)
        mlp_output = torch.relu(mlp_output)
        mlp_output = nn.Dropout(self.dropout)(mlp_output)

        ####### MLP Feature processing branch #######
        user_features_processed = self.user_features_fc(user_features)              # dimension: (batch_size, embedding_dim)
        user_features_processed = torch.relu(user_features_processed)
        user_features_processed = nn.Dropout(self.dropout)(user_features_processed)

        video_features_processed = self.video_features_fc(video_features)           # dimension: (batch_size, embedding_dim)
        video_features_processed = torch.relu(video_features_processed)
        video_features_processed = nn.Dropout(self.dropout)(video_features_processed)

        ####### Combine GMF, MLP, and additional features #######
        combined_input = torch.cat([gmf_output, mlp_output, user_features_processed, video_features_processed], dim=-1)
        combined_output = self.fc1_combined(combined_input)
        # if self.training:
        #     combined_output = nn.BatchNorm1d(128)(combined_output)
        combined_output = torch.relu(combined_output)
        combined_output = nn.Dropout(self.dropout)(combined_output)

        combined_output = self.fc2_combined(combined_output)
        combined_output = torch.relu(combined_output).squeeze()

        ######## Apply decay factors based on interaction time and video age #######
        decay_weight = self.calculate_exponential_weight(interaction_days, video_age_days)
        final_output = combined_output * decay_weight

        return final_output
    
    def calculate_exponential_weight(self, interaction_days, video_age_days):
        """
        Returns the decay weight based on the defined decay constant and time since the transaction was made.
        """
        decay_interaction = torch.exp(-self.alpha * interaction_days)
        decay_video_age = torch.exp(-self.beta * video_age_days)
        
        return decay_interaction * decay_video_age

### Creating the Recommendation System

In [ ]:
class KuaiShou_NCF_RecSys:
    def __init__(self, dataset: KuaiShouDataset, model: nn.Module, embedding_dim: int, dropout: float, alpha: float, beta: float):
        self.dataset = dataset
        self.num_users = len(dataset.user_encoder.classes_)
        self.num_videos = len(dataset.video_encoder.classes_)
        self.num_user_features = len(dataset.user_feature_cols)
        self.num_video_features = len(dataset.video_feature_cols)
        
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # Move model to GPU if available
        
        # Initialise the model
        self.model: nn.Module = model(self.num_users, self.num_videos, embedding_dim, self.num_user_features, self.num_video_features, dropout, alpha, beta)

    def train(self, batch_size, num_epochs, lr, criterion, optimizer):
        # Initialise the DataLoader
        train_loader = DataLoader(self.dataset, batch_size=batch_size, shuffle=True)

        self.model.to(self.device)
        print(f"Model moved to {self.device}")

        # Optimizer and loss function
        optimizer = optimizer(self.model.parameters(), lr=lr)
        criterion = criterion

        # Training loop
        for epoch in range(num_epochs):
            self.model.train()
            total_loss = 0
            
            for user_idx, video_idx, user_features, video_features, watch_ratio in train_loader:
                user_idx, video_idx, user_features, video_features, watch_ratio = user_idx.to(self.device), video_idx.to(self.device), user_features.to(self.device), video_features.to(self.device), watch_ratio.to(self.device)
                
                # Get the interaction days and video age
                interaction_days = self.dataset.get_interaction_age(user_idx, video_idx)
                video_age = self.dataset.get_video_age(video_idx)
                
                # Forward pass
                optimizer.zero_grad()
                outputs = self.model(user_idx, video_idx, user_features, video_features, interaction_days, video_age)
                loss = criterion(outputs, watch_ratio)

                # Backward pass and optimization
                loss.backward()
                optimizer.step()

                # Accumulate loss for reporting
                total_loss += loss.item()

            # Print loss for each epoch
            avg_loss = total_loss / len(train_loader)
            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

    def predict(self):
        """
        Generates a dataframe with predicted watch ratios for each user-video pair.
        """
        self.model.eval()  # Set model to evaluation mode
    
        # Initialise the watch_ratio_matrix (num_users x num_videos)
        watch_ratio_df = pd.DataFrame(columns=['user_id', 'video_id', 'watch_ratio'])
        
        # Iterate through all user-video pairs
        # for user_idx in range(2):
        for user_idx in range(self.num_users):
            if user_idx % 50 == 0:  # Print progress
                print(f"Predicting for user {user_idx}/{self.num_users}")
            for video_idx in range(self.num_videos):
                user_tensor = torch.tensor([user_idx], dtype=torch.long)
                video_tensor = torch.tensor([video_idx], dtype=torch.long)
                
                user_feature_tensor = self.dataset.user_features[user_idx].unsqueeze(0)  # (1, num_user_features)
                video_feature_tensor = self.dataset.video_features[video_idx].unsqueeze(0)  # (1, num_video_features)

                # Here, we set the interaction days to 0 as we are to watch_ratio at the current moment
                interaction_days = torch.tensor([0], dtype=torch.float32)
                video_age = self.dataset.get_video_age(video_tensor)

                # Get the predicted watch ratio
                with torch.no_grad():
                    predicted_watch_ratio = self.model(user_tensor, video_tensor, user_feature_tensor, video_feature_tensor, interaction_days, video_age)
                
                # Add the predicted watch ratio to the watch_ratio_df
                watch_ratio_df = pd.concat([watch_ratio_df, pd.DataFrame({'user_id': [user_idx], 'video_id': [video_idx], 'watch_ratio': [predicted_watch_ratio.item()]})], ignore_index=True)
                
        # Inverse transform the user and video indices to their original values
        watch_ratio_df['user_id'] = self.dataset.inverse_transform_user_ids(watch_ratio_df['user_id'].astype(int))
        watch_ratio_df['video_id'] = self.dataset.inverse_transform_video_ids(watch_ratio_df['video_id'].astype(int))

        return watch_ratio_df
    
    def get_parameters(self):
        """
        Returns the model parameters.
        """
        return self.model.state_dict()

### Fitting the Training Data to the Model and Generating Predictions

In [131]:
# Define the columns for user and video features in the user-item interaction data
user_cols = ['is_lowactive_period',
             'is_live_streamer', 'is_video_author', 'follow_user_num',
             'fans_user_num', 'friend_user_num', 'register_days', 'is_new_user',
             'total_connections', 'is_content_creator', 'hour', 'day_of_week',
             'watch_frequency', 'is_weekend_interaction', 'is_weekend',
             'count_afternoon_views', 'count_evening_views', 'count_midnight_views', 'count_morning_views', 
             'avg_daily_watch_time', 
             'user_active_degree_full_active', 'user_active_degree_high_active', 'user_active_degree_middle_active', 
             'time_period_afternoon', 'time_period_evening', 'time_period_midnight', 'time_period_morning'
            ]
video_cols = ['video_duration', 'show_cnt', 'play_cnt', 
              'like_cnt', 'comment_cnt', 'share_cnt', 'follow_cnt', 'collect_cnt', 
              'News_Politics', 'Auto_Tech', 'Lifestyle', 'Sports_Fitness', 'Entertainment', 'Culture', 'Others',
            ]

In [157]:
def train_and_predict(hyperparameters: dict, train_data: pd.DataFrame, **kwargs):
    cluster = kwargs.get('cluster', None)

    # Set seed for reproducibility
    torch.manual_seed(0)

    BATCH_SIZE = hyperparameters['batch_size']
    NUM_EPOCHS = hyperparameters['num_epochs']
    LEARNING_RATE = hyperparameters['lr']
    EMBEDDING_DIM = hyperparameters['embedding_dim']
    DROPOUT = hyperparameters['dropout']
    ALPHA = hyperparameters['alpha']
    BETA = hyperparameters['beta']

    # Loss function and optimizer
    criterion = nn.MSELoss()
    optimiser = optim.Adam

    print(f"----- Training {'' if cluster == None else f'for cluster {cluster} '}-----")

    # Create the dataset
    dataset_train = KuaiShouDataset(train_data, 'user_id', 'video_id', user_cols, video_cols, 'watch_ratio', interaction_age_dict, video_age_dict)

    # Initialise the NCF model
    print("Initialising...")
    ncf_rec_sys = KuaiShou_NCF_RecSys(dataset_train, NCF, EMBEDDING_DIM, DROPOUT, ALPHA, BETA)

    # Train on data
    ncf_rec_sys.train(BATCH_SIZE, NUM_EPOCHS, LEARNING_RATE, criterion, optimiser)

    # Generate predictions
    print("Generating predictions...")
    predictions_df = ncf_rec_sys.predict()
    
    print("Complete!")
    return cluster, predictions_df

##### Example: Fitting to Cluster 0

In [ ]:
params = {
    'batch_size': 512,
    'num_epochs': 10,
    'lr': 0.001,
    'embedding_dim': 64,
    'dropout': 0.3,
    'alpha': 0.01,  # Decay constant for days since interaction (days)
    'beta': 0.01    # Decay constant for video age (days)
}

cluster = 0
train_cluster = train_processed[train_processed['cluster'] == cluster]

cluster, cluster_0_predictions = train_and_predict(params, train_cluster, **{'cluster': 0})

----- Training for cluster 0 -----
Initialising...
Model moved to cpu
Epoch [1/10], Loss: 0.4180
Epoch [2/10], Loss: 0.3756
Epoch [3/10], Loss: 0.3633
Epoch [4/10], Loss: 0.3573
Epoch [5/10], Loss: 0.3537
Epoch [6/10], Loss: 0.3511
Epoch [7/10], Loss: 0.3483
Epoch [8/10], Loss: 0.3458
Epoch [9/10], Loss: 0.3435
Epoch [10/10], Loss: 0.3405
Generating predictions...
Predicting for user 0/269
Predicting for user 50/269
Predicting for user 100/269
Predicting for user 150/269
Predicting for user 200/269
Predicting for user 250/269


ValueError: too many values to unpack (expected 1)

In [ ]:
cluster_0_predictions

,user_id,video_id,watch_ratio,cluster
0,14,103,1.287236,0
1,14,109,1.725524,0
2,14,120,1.315136,0
3,14,122,1.367776,0
4,14,128,0.697545,0
...,...,...,...,...
634835,7162,10099,1.488036,0
634836,7162,10105,0.845313,0
634837,7162,10120,1.086396,0
634838,7162,10122,0.990733,0


### Grid Search for Hyperparameter Tuning

In [ ]:
import itertools

In [ ]:
# Is there a way to parallelise this?
def train_by_cluster_and_without(params: dict, train_data: pd.DataFrame, train_by_cluster: bool = True, train_without_clustering: bool = False):
    param_str = '_'.join([f'{key}{val}' for key, val in params.items()])

    # Train for each cluster
    if train_by_cluster:
        cluster_predictions = {}
        for cluster in sorted(train_data['cluster'].unique()):
            train_cluster = train_data[train_data['cluster'] == cluster]

            cluster, predictions_df = train_and_predict(params, train_cluster, **{'cluster': cluster})
            cluster_predictions[cluster] = predictions_df

            # Include parameters in the output file name
            output_file = root + f'results/cluster{cluster}_{param_str}.csv'
            predictions_df.to_csv(output_file, index=False)

            print(f'Predictions for cluster {cluster} saved to {output_file}')
        
        # Save combined predictions to a single file
        watch_ratio_predictions_df = pd.DataFrame()
        for cluster, df in cluster_predictions.items():
            cluster_predictions_df = df
            cluster_predictions_df['cluster'] = cluster
            
            watch_ratio_predictions_df = pd.concat([watch_ratio_predictions_df, cluster_predictions_df])
        
        # Include parameters in the output file name
        output_file = root + f'results/w_clustering_{param_str}.csv'
        watch_ratio_predictions_df.to_csv(output_file, index=False)
        print(f'Predictions with segmentation saved to {output_file}')
    
    # Train without clustering
    if train_without_clustering:
        _, predictions_df = train_and_predict(params, train_data)

        # Include parameters in the output file name
        output_file = root + f'results/wo_clustering_{param_str}.csv'
        predictions_df.to_csv(output_file, index=False)
        print(f'Predictions without segmentation saved to {output_file}')

In [ ]:
# hyperparameters = {
#     'batch_size': [256, 512],
#     'num_epochs': [10],
#     'lr': [0.001, 0.01],
#     'embedding_dim': [32, 64],
#     'dropout': [0.3, 0.5],
#     'alpha': [0.01, 0.05],
#     'beta': [0.01, 0.05]
# }
hyperparameters = {
    'batch_size': [512],
    'num_epochs': [1],
    'lr': [0.001],
    'embedding_dim': [64],
    'dropout': [0.3],
    'alpha': [0.01],
    'beta': [0.01]
}

# Generate all possible combinations of hyperparameters
param_combinations = list(itertools.product(*hyperparameters.values()))

# Train for each combination of hyperparameters
for params in param_combinations:
    params_dict = {key: val for key, val in zip(hyperparameters.keys(), params)}
    train_by_cluster_and_without(params_dict, train_processed, True, True)

----- Training for cluster 0 -----
Initialising...
Model moved to cpu
Epoch [1/1], Loss: 0.4180
Generating predictions...
Predicting for user 0/269
Complete!
Predictions for cluster 0 saved to ../KuaiRec 2.0/results/cluster0_batch_size512_num_epochs1_lr0.001_embedding_dim64_dropout0.3_alpha0.01_beta0.01.csv
----- Training for cluster 1 -----
Initialising...
Model moved to cpu
Epoch [1/1], Loss: 0.2232
Generating predictions...
Predicting for user 0/419
Complete!
Predictions for cluster 1 saved to ../KuaiRec 2.0/results/cluster1_batch_size512_num_epochs1_lr0.001_embedding_dim64_dropout0.3_alpha0.01_beta0.01.csv
----- Training for cluster 2 -----
Initialising...
Model moved to cpu
Epoch [1/1], Loss: 0.2729
Generating predictions...
Predicting for user 0/345
Complete!
Predictions for cluster 2 saved to ../KuaiRec 2.0/results/cluster2_batch_size512_num_epochs1_lr0.001_embedding_dim64_dropout0.3_alpha0.01_beta0.01.csv
----- Training for cluster 3 -----
Initialising...
Model moved to cpu
Epo

In [ ]:
# class NCF(nn.Module):
#     def __init__(self, num_users, num_videos, embedding_dim, num_user_features, num_video_features):
#         super(NCF, self).__init__()
        
#         # GMF Components for embeddings
#         self.user_embeddings_gmf = nn.Embedding(num_users, embedding_dim)
#         self.video_embeddings_gmf = nn.Embedding(num_videos, embedding_dim)

#         # MLP Components for embeddings
#         self.user_embeddings_mlp = nn.Embedding(num_users, embedding_dim)
#         self.video_embeddings_mlp = nn.Embedding(num_videos, embedding_dim)

#         # MLP layers for user and video embeddings
#         self.fc1_mlp = nn.Linear(2 * embedding_dim + num_user_features + num_video_features, 128)
#         self.fc2_mlp = nn.Linear(128, 64)

#         # Final layers combining GMF, MLP for embeddings, and additional features
#         self.fc1_combined = nn.Linear(embedding_dim + 64, 128)
#         self.fc2_combined = nn.Linear(128, 1)

#     def forward(self, user_id, video_id, user_features, video_features, dropout=0.5):
#         ##### GMF Embedding branch
#         user_emb_gmf = self.user_embeddings_gmf(user_id)
#         video_emb_gmf = self.video_embeddings_gmf(video_id)
#         gmf_output = user_emb_gmf * video_emb_gmf   # dimension: (batch_size, embedding_dim)

#         ##### MLP Embedding branch
#         user_emb_mlp = self.user_embeddings_mlp(user_id)
#         video_emb_mlp = self.video_embeddings_mlp(video_id)

#         mlp_input = torch.cat([user_emb_mlp, video_emb_mlp, user_features, video_features], dim=-1) 

#         # First fully connected layer with BatchNorm and ReLU
#         mlp_output = self.fc1_mlp(mlp_input)
#         # if self.training:
#         #     mlp_output = nn.BatchNorm1d(128)(mlp_output)
#         mlp_output = torch.relu(mlp_output)
#         mlp_output = nn.Dropout(dropout)(mlp_output)

#         # Second fully connected layer with BatchNorm and ReLU
#         mlp_output = self.fc2_mlp(mlp_output)
#         # if self.training:
#         #     mlp_output = nn.BatchNorm1d(64)(mlp_output)
#         mlp_output = torch.relu(mlp_output)
#         mlp_output = nn.Dropout(dropout)(mlp_output)

#         ##### Combine GMF, MLP
#         combined_input = torch.cat([gmf_output, mlp_output], dim=-1)
#         combined_output = self.fc1_combined(combined_input)
#         # if self.training:
#         #     combined_output = nn.BatchNorm1d(128)(combined_output)
#         combined_output = torch.relu(combined_output)
#         combined_output = nn.Dropout(dropout)(combined_output)

#         ##### Scale wach_ratio to range [0, 5]
#         combined_output = self.fc2_combined(combined_output)
#         combined_output = torch.relu(combined_output)

#         return combined_output.squeeze()